In [ ]:
import tensorflow as tf
import keras as ks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [ ]:
print(f"Tensorflow Version {tf.__version__}")
print(f"Keras Version {ks.__version__}")

In [ ]:
check_list = ['GPU','CPU']

for ob in check_list:
    c = tf.config.list_physical_devices(ob)
    if len(c) > 0:
        print(f"{ob} is avaliable ")
    else:
        print(f"{ob} is not avaliable")

In [ ]:
mnist = tf.keras.datasets.mnist

In [ ]:
mnist

In [ ]:
(X_train, Y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
print(f"Data Type of X_train {X_train.dtype}")
print(f"Data Type of X_train {Y_train.dtype}")

In [ ]:
x_valid, x_train = X_train[:5000]/255, X_train[5000:]/255
y_valid, y_train = Y_train[:5000]/255, Y_train[5000:]
x_test = X_test/255

In [ ]:
# Train 50000
# Validation - 5000
# Test - 10000

In [ ]:
plt.imshow(x_train[1],cmap='binary')

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(x_train[1],annot=True,cmap='binary')

In [ ]:
LAYERS = [
    ks.layers.Flatten(input_shape=[28,28],name='inputLayer'),
    ks.layers.Dense(300,activation='relu',name='hiddenLayer1'),
    ks.layers.Dense(10,activation='softmax',name='outputLayer') 
]
model_clf = ks.models.Sequential(LAYERS)

In [ ]:
model_clf.layers

In [ ]:
model_clf.summary()

In [ ]:
hidden1 = model_clf.layers[1]
hidden1.name

In [ ]:
hidden2 = model_clf.layers[2]
print(f"hidden2 name:{hidden2.name}")

In [ ]:
hidden1.get_weights()[0].shape

In [ ]:
hidden2.get_weights()[0].shape

In [ ]:
weights, biases = hidden1.get_weights()

In [ ]:
weights.shape

In [ ]:
biases.shape

In [ ]:
LOSS_FUNCTION = "sparse_categorical_crossentropy"
OPTIMIZER = "SGD"
METRICS = ['accuracy']

model_clf.compile(loss=LOSS_FUNCTION, optimizer=OPTIMIZER,metrics=METRICS)

## Tensorboard Callback Function

In [ ]:
import time
import os
def get_log_path(log_dir='logs/fit'):
    fileName = time.strftime("log_%Y_%m_%d_%H_%M_%S")
    logs_path = os.path.join(log_dir,fileName)
    print(f"Saving logs at {logs_path}")
    return logs_path

log_dir = get_log_path()
tb_cb = ks.callbacks.TensorBoard(log_dir=log_dir)

## Early Stopping Callback

In [ ]:
early_stopping_cb = ks.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

## Model Checkpointing Callback

In [ ]:
CKPT_path = "Model_ckpt.h5"
checkpointing_cb = ks.callbacks.ModelCheckpoint(CKPT_path, save_best_only=True)

In [ ]:
# Training
Epochs = 30
validation_set = (x_valid, y_valid)
history = model_clf.fit(x_train, y_train, epochs=Epochs, validation_data=validation_set, batch_size=32, callbacks=[tb_cb, early_stopping_cb, checkpointing_cb])

In [ ]:
ckpt_model = ks.models.load_model("Model_ckpt.h5")
ckpt_model.compile(optimizer=OPTIMIZER, loss=LOSS_FUNCTION, metrics=METRICS)

history = ckpt_model.fit(x_train,y_train, epochs=10, validation_data=validation_set,batch_size=32,callbacks=[tb_cb, early_stopping_cb, checkpointing_cb])

#### Saving the Model

In [ ]:
import time
import os

def save_model_path(MODEL_dir='TRAINED_MODEL'):
    os.makedirs(MODEL_dir,exist_ok=True)
    fileName= time.strftime("Model_%Y_%m_%d_%H_%M_%S_.h5")
    model_path = os.path.join(MODEL_dir,fileName)
    print(f"Model {fileName} will be saved at {model_path}")
    return model_path

In [ ]:
UNIQUE_PATH = save_model_path()
UNIQUE_PATH

In [ ]:
ks.models.save_model(model_clf,UNIQUE_PATH)

In [ ]:
history.params

In [ ]:
pd.DataFrame(history.history)

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
model_clf.evaluate(x_test,y_test)

In [ ]:
x_new = x_test[:3]

In [ ]:
actual = y_test[:3]

In [ ]:
actual

In [ ]:
y_prob = model_clf.predict(x_new)
y_prob.round(3)

In [ ]:
y_pred = np.argmax(y_prob,axis=-1)

In [ ]:
y_pred

In [ ]:
for data,pred,actual_data in zip(x_new, y_pred, actual):
    plt.imshow(data,cmap='binary')
    plt.title(f'Predicted {pred} and Actual {actual_data}')
    plt.axis("off")
    plt.show()


In [ ]:
path_name = "TRAINED_MODEL/Model_2025_03_23_22_25_05_.h5"
loaded_model = ks.models.load_model(path_name)
loaded_model.evaluate(x_test,y_test)

In [ ]:
result = loaded_model.predict(x_new)
y_pred = np.argmax(result,axis=-1)

In [ ]:
y_pred

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit